In [14]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt

#Load previous camera calibration parameter
dist_pickle = pickle.load( open("./camera_cal/calibration_pickle.p","rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary_output

def color_threshold(image, sthresh=(0,255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0])&(s_channel <= sthresh[1])] = 1

    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v_channel = hls[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= vthresh[0])&(v_channel <= vthresh[1])] = 1

    output = np.zeros_like(s_channel)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output



for idx, fname in enumerate(images):
    #read and undistor images
    img = cv2.imread(fname)
    
    img = cv2.undistort(img,mtx,dist,None,mtx)

    # Process image and generate binary pixel of interests
    preprocessImage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    c_binary = color_threshold(img, sthresh=(100,255),vthresh=(50,255))
    preprocessImage[((gradx == 1)&(grady == 1)|(c_binary == 1))] = 255

    # Defining perspective transformation area
    height = img.shape[0]
    width = img.shape[1]
    img_size = (width, height)
  
    top_left_src = (563, 470)
    bottom_left_src = (220, 700)
    top_left_dst = (300,300)
    bottom_left_dst = (300,720)

    src = np.float32([[top_left_src[0],top_left_src[1]], 
    [bottom_left_src[0],bottom_left_src[1]],
    [width - bottom_left_src[0],bottom_left_src[1]],
    [width - top_left_src[0],top_left_src[1]]]) 

    dst = np.float32([[top_left_dst[0],top_left_dst[1]], 
    [bottom_left_dst[0],bottom_left_dst[1]],
    [width - bottom_left_dst[0],bottom_left_dst[1]],
    [width - top_left_dst[0],top_left_dst[1]]])

    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(preprocessImage,M,img_size,flags=cv2.INTER_LINEAR)

    result = warped

    write_name = './test_images/tracked'+str(idx+1)+'.jpg'
    cv2.imwrite(write_name, result)
